In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
# unzip the files
import zipfile

datasets = ("sample_submission.csv", "test.csv", "test_labels.csv", "train.csv")

for d in datasets:
    with zipfile.ZipFile("../input/jigsaw-toxic-comment-classification-challenge/"+d+".zip","r") as z:
        z.extractall(".")

In [3]:
test = pd.read_csv("/kaggle/working/test.csv")
train = pd.read_csv("/kaggle/working/train.csv")
sample_submission = pd.read_csv("/kaggle/working/sample_submission.csv")
test_labels = pd.read_csv("/kaggle/working/test_labels.csv")

# Text pre-processing

In [4]:
import re
from bs4 import BeautifulSoup


def pre_process(text):
    text = BeautifulSoup(text).get_text()
    # fetch alphabetic characters

    text = re.sub("[^a-zA-Z]", " ", text)
    # convert text to lower case
    text = text.lower()
    # split text into tokens to remove whitespaces
    tokens = text.split()
    return " ".join(tokens)

train['comment_text'] = train['comment_text'].map(pre_process)

/tmp/ipykernel_33/2397898667.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text).get_text()
/tmp/ipykernel_33/2397898667.py:6: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text).get_text()


In [5]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
train['comment_text'] = train['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train['word_count'] = train['comment_text'].apply(lambda x: len(str(x).split(" ")))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
from nltk.stem.snowball import SnowballStemmer

#stemming
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

train['comment_text'] = train['comment_text'].apply(stemming)

## tfidf

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
countvectorizer = CountVectorizer(analyzer='word' , stop_words='english')
tfidfvectorizer = TfidfVectorizer(analyzer='word' , stop_words='english',)

In [8]:
corpus = train.comment_text
X = tfidfvectorizer.fit_transform(corpus)
X.shape

(159571, 131949)

In [9]:
tfidf_tokens = tfidfvectorizer.get_feature_names_out()

In [10]:
y = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

### sample 10,000 data to demo and get the model to run

In [11]:
import random
from random import sample
random.seed(4)
i = sample(range(X.shape[0]), 10000)

In [12]:
X_new = X[i]
X_new = X_new.toarray()
y_new = y.iloc[i]

## stratified sampling to handle imbalanced data for multilabel classification
- useful [reference](https://github.com/scikit-multilearn/scikit-multilearn/issues/194) to get data into right format

In [13]:
from skmultilearn.model_selection import iterative_train_test_split
x_train, y_train, x_test, y_test = iterative_train_test_split(X_new, y_new.values, test_size = 0.1)

# Model

In [15]:
# Imports
from sklearn.datasets import make_multilabel_classification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [16]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [17]:
# model config
N_FEATURES = X.shape[1]
N_CLASSES = len(labels) 
N_EPOCHS = 50
RANDOM_STATE = 35
BATCH_SIZE = 250
VERBOSITY = 1
VALIDATION_SPLIT = 0.2

In [22]:
model = Sequential()
# layers get narrower, larger details captured first
model.add(Dense(64, activation='relu', input_dim=N_FEATURES))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
# sigmoid layer in last layer to generate prediction value within [0,1]
model.add(Dense(N_CLASSES, activation='sigmoid'))

In [23]:
from tensorflow import keras

In [24]:
# Compile the model
model.compile(loss=binary_crossentropy, # for multi-label classification
              optimizer=Adam(),
              metrics=['accuracy', keras.metrics.AUC(name='auc')])

In [25]:
# Fit data to model
model.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=N_EPOCHS,
          verbose=VERBOSITY,
          validation_split=VALIDATION_SPLIT)

Epoch 1/50
29/29 [==============================] - 11s 330ms/step - loss: 0.6343 - accuracy: 5.5556e-04 - auc: 0.5019 - val_loss: 0.5454 - val_accuracy: 0.0000e+00 - val_auc: 0.5994
Epoch 2/50
29/29 [==============================] - 8s 264ms/step - loss: 0.4159 - accuracy: 0.0000e+00 - auc: 0.5904 - val_loss: 0.3169 - val_accuracy: 0.0000e+00 - val_auc: 0.6334
Epoch 3/50
29/29 [==============================] - 7s 258ms/step - loss: 0.1982 - accuracy: 0.0000e+00 - auc: 0.7221 - val_loss: 0.1750 - val_accuracy: 0.0022 - val_auc: 0.7812
Epoch 4/50
29/29 [==============================] - 9s 317ms/step - loss: 0.1085 - accuracy: 0.6097 - auc: 0.8751 - val_loss: 0.1434 - val_accuracy: 0.9950 - val_auc: 0.8783
Epoch 5/50
29/29 [==============================] - 13s 436ms/step - loss: 0.0820 - accuracy: 0.9940 - auc: 0.9453 - val_loss: 0.1192 - val_accuracy: 0.9950 - val_auc: 0.9299
Epoch 6/50
29/29 [==============================] - 8s 263ms/step - loss: 0.0597 - accuracy: 0.9940 - auc: 0

### Evlauate on test dataset

In [26]:
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {round(score[0], 4)} / AUC: {round(score[2],4)}')

Test loss: 0.2087 / AUC: 0.8205
